In [241]:
import torch
import pandas as pd
import os
import numpy as np
import json
import h5py
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import time
import sys
import pprint
from collections import Counter,defaultdict
from itertools import chain

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# TODO: find scientific reference that also claims Snowball is better
# alternatively: http://www.nltk.org/howto/stem.html claims this already.
from nltk.stem import SnowballStemmer, PorterStemmer

# check if stopword corpus is available on your system
try:
    _ = stopwords.words('english')
except:
    nltk.download('stopwords')
    
try:
    _ = WordNetLemmatizer().lemmatize('test')
except:
    nltk.download('wordnet')

class DialogDataset(Dataset):
    
    def __init__(self, json_data, image_features, img2feat, transform=None):
        
        with open(img2feat, 'r') as f:
            self.img2feat = json.load(f)['IR_imgid2id']
            
        self.img_features = np.asarray(h5py.File(image_features, 'r')['img_features'])
        self.json_data = pd.read_json(json_data, orient='index')
        self.corpus = self.filter_document(self.get_words())
        self.vocab = list(set(self.corpus))
        self.w2i = {word : i for i, word in enumerate(self.vocab)}
        
    # collect all the words from dialogs and 
    # captions and use them to create embedding map
    def get_words(self):
        words = []
        for idx in range(len(self)):
            item = self.json_data.iloc[idx]

            # Flatten dialog and add caption into 1d array
            dialog = [word for line in item.dialog for word in line[0].split()]
            dialog.extend(item.caption.split(' '))

            words.append(dialog)
            
        return list(chain.from_iterable(words))
    
    def make_context_vector(self, context):
        idxs = [self.w2i[w] for w in context]
        tensor = torch.LongTensor(idxs)
        return autograd.Variable(tensor)
    
    def filter_document(self, document):
        """Filter list of words based on some conventinal methods, like removing stopwords and
        lemmatization"""
        
#         # Remove stop words
        stop_words = stopwords.words('english')
        document = filter(lambda x: x not in stop_words, document)
        
        # Lemmatize
#         lemmatizer = WordNetLemmatizer()
#         document = map(lemmatizer.lemmatize, document)
        
#         # Stem
        stemmer = SnowballStemmer('english')
        document = map(stemmer.stem, document)
        
        return document
        
        

    def __len__(self):
        return len(self.json_data)

    def __getitem__(self, idx):
        item = self.json_data.iloc[idx]

        # Flatten dialog and add caption into 1d array
        dialog = [word for line in item.dialog for word in line[0].split()]
        dialog.extend(item.caption.split(' '))
        dialog = self.filter_document(dialog)
        dialog = self.make_context_vector(dialog)

        img_ids = np.array(item.img_list)
        img_features = [self.img_features[idx] for idx in map(lambda x: self.img2feat[str(x)], img_ids)]
        img_features = np.array(img_features)
        img_features = Variable(torch.FloatTensor(img_features))
        
        target_idx = item.target
        
        return {'dialog' : dialog, 'img_features': img_features}, target_idx

In [242]:
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.Sigmoid()
        

    def forward(self, inputs):
        # i believe .view() is useless here because the sum already produces a 1xEMB_DIM vector
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

In [243]:
class MaxEnt(torch.nn.Module):
    
    def __init__(self, text_module, vocab_size, img_size):
        super(MaxEnt, self).__init__()

        self.text_module = text_module
        self.linear = nn.Linear(vocab_size + img_size, 1)
        
    def forward(self, inp):
        dialog = inp['dialog']
        all_img_features = inp['img_features']
        
        text_features = self.text_module(dialog)
        text_features = text_features.expand((all_img_features.size(0), text_features.size(1)))
    
        concat = torch.cat((all_img_features, text_features), 1)
        
        scores = self.linear(concat)
        scores = F.log_softmax(scores.transpose(0, 1))
        
        return scores
    


In [244]:
SAMPLE_EASY = ['Data', 'sample_easy.json']
TRAIN_EASY = ['Data', 'Easy', 'IR_train_easy.json']
VALID_EASY = ['Data', 'Easy', 'IR_val_easy.json']
IMG_FEATURES = ['Data', 'Features', 'IR_image_features.h5']
INDEX_MAP = ['Data', 'Features', 'IR_img_features2id.json']

IMG_SIZE = 2048
EMBEDDING_DIM = 5

torch.manual_seed(1)
# dialog_data = DialogDataset(os.path.join(*SAMPLE_EASY), os.path.join(*IMG_FEATURES), os.path.join(*INDEX_MAP))
%time dialog_data = DialogDataset(os.path.join(*TRAIN_EASY), os.path.join(*IMG_FEATURES), os.path.join(*INDEX_MAP))
valid_data = DialogDataset(os.path.join(*VALID_EASY), os.path.join(*IMG_FEATURES), os.path.join(*INDEX_MAP))

vocab_size = len(dialog_data.vocab)

CPU times: user 1min 30s, sys: 2.8 s, total: 1min 33s
Wall time: 1min 44s


In [246]:
cbow_model = CBOW(vocab_size, EMBEDDING_DIM)
max_ent = MaxEnt(cbow_model, vocab_size, IMG_SIZE)
max_ent(dialog_data[0][0])

Variable containing:
-2.2969 -2.3151 -2.3128 -2.3159 -2.4021 -2.3224 -2.2387 -2.3621 -2.1880 -2.2877
[torch.FloatTensor of size 1x10]

In [251]:
def validate(model, data, loss_func):
    total_loss = 0
    
    for i, (inp, target) in enumerate(data):
        pred = model(inp)
        target = Variable(torch.LongTensor(np.array([target])))
        
        loss = loss_func(pred, target)
        total_loss += loss
        
        if i == 20:
            break
    
    return total_loss / 20

def predict(model, data):
    correct = 0
    
    for i, (inp, target) in enumerate(data):
        pred = model(inp)
        img, idx = torch.max(pred, 1)
        if idx.data[0] == target:
            correct += 1
        
        if i == 20:
            break
    
    return correct

validate(max_ent, valid_data, nn.NLLLoss())

Variable containing:
 2.3980
[torch.FloatTensor of size 1]

In [253]:
EPOCHS = 10

cbow_model = CBOW(vocab_size, EMBEDDING_DIM)
max_ent = MaxEnt(cbow_model, vocab_size, IMG_SIZE)
loss_func = nn.NLLLoss()
optimizer = optim.Adam(max_ent.parameters(), lr=1e-05)
validation_errors = []

for epoch in range(1, EPOCHS + 1):
    total_loss = 0
    for i, (inp, target) in enumerate(dialog_data):
        
        # Make prediction
        pred = max_ent(inp)
        target = Variable(torch.LongTensor(np.array([target])))

        # Calculate loss
        loss = loss_func(pred, target)
        total_loss += loss.data[0]
        
        # Zero out gradients
        max_ent.zero_grad()
        
        # Backpropagate and update parameters
        loss.backward()
        optimizer.step()
        
        if i == 500:
            break
    
    total_loss = total_loss / 500
    print("Epoch {}: {}".format(epoch, total_loss))
    print("Predicted {}/20 samples correctly".format(predict(max_ent, valid_data)))
    
    val = validate(max_ent, valid_data, loss_func)
    validation_errors.append(val.data[0])
        
    
print(validation_errors)

Epoch 1: 2.309984299182892
Predicted 1/20 samples correctly
Epoch 2: 2.292324097156525
Predicted 7/20 samples correctly
Epoch 3: 2.2774351871013643
Predicted 7/20 samples correctly
Epoch 4: 2.2647252728939056
Predicted 8/20 samples correctly
Epoch 5: 2.253857351064682
Predicted 9/20 samples correctly
Epoch 6: 2.244542367696762
Predicted 11/20 samples correctly
Epoch 7: 2.2365342650413513
Predicted 11/20 samples correctly
Epoch 8: 2.2296245720386505
Predicted 11/20 samples correctly
Epoch 9: 2.2236373114585875
Predicted 11/20 samples correctly
Epoch 10: 2.218424485683441
Predicted 11/20 samples correctly
[2.426304578781128, 2.4022343158721924, 2.38154935836792, 2.3637640476226807, 2.3484585285186768, 2.335270881652832, 2.3238911628723145, 2.314053773880005, 2.3055310249328613, 2.298130512237549]
